In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import random
from random import randint
from numba import jit
from utility import *
from tqdm import tqdm
from sklearn.datasets import make_blobs

Implementazione dell'algoritmo K-means:

In [ ]:
def k_means(points, K):
    N = points.shape[0]
    dimension = points.shape[1]
    #inizializzazione dei centroidi in corrispondenza di K punti 
    centroids = np.zeros((K,dimension))
    clusters = np.zeros(N)
    choices = np.arange(N)
    for i in range(K):
        choice = np.random.randint(len(choices))
        centroids[i] = points[choice].copy()#centroids_list[i]
        choices = np.delete(choices, choice)
    finito = False
    iter = 0
    while(finito == False):
        for i in range(N):
            dist = -1
            centroid = -1
            for c in range(K):
                dist_c = np.linalg.norm(centroids[c]-points[i])
                if(dist_c < dist or dist == -1):
                    centroid = c
                    dist = dist_c
            clusters[i] = centroid        
        old_centroids = centroids.copy()

        for i in range(K): #calcolo nuovi centroidi
            centroids[i] = np.zeros(dimension)
            n_elem = 0
            for j in range(N):
                if clusters[j] == i:
                    n_elem = n_elem+1
                    centroids[i] = centroids[i] + points[j]
            if(n_elem == 0):
                centroids[i] = points[np.random.randint(len(choices))].copy()
            else:
                centroids[i] = centroids[i] / n_elem

        iter = iter + 1
        if((old_centroids == centroids).all()):#criterio di arresto
            finito = True
                
    return clusters

Creazione di un'istanza di esempio con 500 punti in R^2 e 5 cluster 

In [ ]:
points, centroids = make_blobs(n_samples=500, centers=5, n_features=2, random_state=2)

N = len(points)
K = 5

sol = k_means(points, K)

print("{:.5E}".format(squared_inner_distance(sol, points, K)))#mostra soltanto 5 cifre significative

In [ ]:
printR2sol(points, sol, K)

In [ ]:
n_points = [500,100,200,500,1400,1000,400,700,300,800,5000,10000]
n_clusters = [7,4,3,2,3,2,4,5,4,6,4,5]
dim_points = [32,16,24,10,20,16,12,20,64,32,3,3]

vals = []

for test in tqdm(range(1,13)):
    dirname = os.path.dirname("__file__")
    filename = os.path.join(dirname, f'./benchmark/benchmark{test}.txt')
    points = load_points(filename)
    N = len(points)
    K = n_clusters[test-1]
    best_val = -1
    for i in range(20): 
        sol = k_means(points, K)
        val = squared_inner_distance(sol, points, K)
        if(best_val == -1 or best_val > val):
            best_val = val
    vals.append(val)


with open("risultatiKMEANS.txt", 'w') as file:
    file.write("K-means:\n")
    file.write(str(vals))

In [ ]:
#Identificare il numero ottimo di cluster:

points, centroids = make_blobs(n_samples=500, centers=6, n_features=2, random_state=2, cluster_std = 0.2)

plt.scatter(points[:,0], points[:,1])
plt.show()


cluster_val = []
for i in tqdm(range(5,6)):
    best_val = -1
    for i in range(10): 
        sol = k_means(points, i)
        val = squared_inner_distance(sol, points, i)
        if(best_val == -1 or best_val > val):
            best_val = val
            print(best_val)
    cluster_val.append(best_val)

plt.plot(range(2,20), cluster_val)